# Tensorflow 的可视化 Tensorboard

在上一篇notebook中，我们为大家展示了一个简单的基于Tensorflow的卷积神经网络。Tensorflow的可视化--tensorboard是tensorflow最主要的优点之一。不管多么复杂的神经网络结构，tensorboard都可以将其结构以graph的形式展现出来，并且标注出data的流向，tensorflow也因此得名。

今天，我们将基于上一篇的CNN code，为大家简单说明怎么应用并且调用Tensorboard.


首先，我们先看一下改过的code。绝大多数code和上一篇的CNN一样，我们这里其实只做了一小部分的修改。我们会逐一将改动的地方加以说明。

In [ ]:
import input_data
import tensorflow as tf

def weight_variable(shape, names):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial, name = names)

def bias_variable(shape, names):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial, name = names)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
  
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)
  
  
###########################################################

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

##########################################################

graphCNN = tf.Graph()
with graphCNN.as_default():

    # define x and y_
    x = tf.placeholder("float", shape=[None, 784])
    y_ = tf.placeholder("float", shape=[None, 10])
    
    x_image = tf.reshape(x, [-1,28,28,1])
      
    
    with tf.name_scope('CNN1'):
        W_conv1 = weight_variable([5, 5, 1, 32], 'weights1')
        b_conv1 = bias_variable([32], 'bias1')
        variable_summaries(W_conv1)
        variable_summaries(b_conv1)
        
    with tf.name_scope('CNN1-Act'):
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
        h_pool1 = max_pool_2x2(h_conv1)   
        tf.summary.histogram('post_activations1', h_pool1)
    
    
    with tf.name_scope('CNN2'):
        W_conv2 = weight_variable([5, 5, 32, 64], 'weights2')
        b_conv2 = bias_variable([64], 'bias2')
        variable_summaries(W_conv2)
        variable_summaries(b_conv2)
    
    with tf.name_scope('CNN2-Act'):
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
        h_pool2 = max_pool_2x2(h_conv2)
    tf.summary.histogram('post_activations2', h_pool2)
    
    
    with tf.name_scope('FC1'):
        # 7 = 28/2/2
        W_fc1 = weight_variable([7 * 7 * 64, 1024], 'weightFC1')
        b_fc1 = bias_variable([1024], 'biasFC1')
        variable_summaries(W_fc1)
        variable_summaries(b_fc1)
    
    
    with tf.name_scope('FC1-Act'):
        # reshape the h_pool2 for the fully connected layer
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    tf.summary.histogram('post_activations2', h_fc1)

  
    with tf.name_scope('Dropout'):       
        keep_prob = tf.placeholder("float")
        tf.summary.scalar('dropout_keep_probability', keep_prob)
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    
    with tf.name_scope('FC2'):
        W_fc2 = weight_variable([1024, 10], 'weightFC2')
        b_fc2 = bias_variable([10], 'biasFC2')
        variable_summaries(W_fc2)
        variable_summaries(b_fc2)
    
    y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
    
    with tf.name_scope('cross_entropy'):
        cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
    tf.summary.scalar('cross_entropy', cross_entropy)

    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    tf.summary.scalar('accuracy', accuracy)


#######################################################################


with tf.Session(graph=graphCNN) as sess:
        
    summary_op = tf.summary.merge_all()
    
    init = tf.global_variables_initializer()
    
    summary_writer = tf.summary.FileWriter("tmp/tensorflowlogs", sess.graph)  # not /tmp/...
    
    sess.run(init)
    
    for i in range(1000):
        
        batch = mnist.train.next_batch(50)
        
        if i%500 == 0:   
            # notice accuracy.eval; predicting using keep prob 1.0
            train_accuracy = sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})   
            print("step %d, training accuracy %g"%(i, train_accuracy))
        
        # notice train_step.run; trainning using keep prob 0.5
        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
                
        if i%100 == 0:   
           summary_str = sess.run(summary_op, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
           summary_writer.add_summary(summary_str, i)
           summary_writer.flush() 

## 1. 定义一个能可视化Graph

TensorBoard 通过读取 TensorFlow 的事件文件来运行。TensorFlow 的事件文件包括了你会在 TensorFlow 运行中涉及到的主要数据。

所以，第一步，当我们在构建Graph的时候，需要告诉Tensorflow我们想可视化这个Graph。具体code


## tf.summary.scalar

首先，创建你想汇总数据的 TensorFlow Graph，然后再选择你想在哪个节点进行汇总(summary)操作。

比如，假设你正在训练一个卷积神经网络，用于识别 MNISt 标签。你可能希望记录学习速度(learning rate)的如何变化，以及目标函数如何变化。通过向节点附加scalar_summary操作来分别输出学习速度和期望误差。然后你可以给每个 scalary_summary 分配一个有意义的  标签 ，比如  'learning rate'  和  'loss function' 。

或者你还希望显示一个特殊层中激活的分布，或者梯度权重的分布。可以通过分别附加 histogram_summary 运算来收集权重变量和梯度输出。


在TensorFlow中，所有的操作只有当你执行，或者另一个操作依赖于它的输出时才会运行。我们刚才创建的这些节点（summary nodes）都围绕着你的图像：没有任何操作依赖于它们的结果。因此，为了生成汇总信息，我们需要运行所有这些节点。这样的手动工作是很乏味的，因此可以使用tf.merge_all_summaries来将他们合并为一个操作。

然后你可以执行合并命令，它会依据特点步骤将所有数据生成一个序列化的 Summary  protobuf对象。最后，为了将汇总数据写入磁盘，需要将汇总的protobuf对象传递给tf.train.Summarywriter。

 SummaryWriter  的构造函数中包含了参数 logdir。这个 logdir 非常重要，所有事件都会写到它所指的目录下。此外， SummaryWriter  中还包含了一个可选择的参数  GraphDef 。如果输入了该参数，那么 TensorBoard 也会显示你的图像。

现在已经修改了你的图，也有了  SummaryWriter ，现在就可以运行你的神经网络了！如果你愿意的话，你可以每一步执行一次合并汇总，这样你会得到一大堆训练数据。这很有可能超过了你想要的数据量。你也可以每一百步执行一次合并汇总，或者如下面代码里示范的这样。


## tf.summary.merge_all

## summary_writer.add_summary(summary_str, i)
## summary_writer.flush()